In [1]:
# Import required libraries and set folder path
import pandas as pd
import numpy as np
import os,sys,re
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/28_ResearchMatch/')

%run 00.Scripts/01.CitationMaster.py

article_title 0 3389
article_title 1 5915
abstract 0 10153
abstract_article_title 0 10256


C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
00.Scripts/01.CitationMaster.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [ ]:
for th1,th2 in product([0.3],[0.3]):
    # Make selection of recommendation
    test.loc[:,'score']=clf.transform(test.loc[:,idv].fillna(0))[:,1]
    test.loc[:,'score_rank']=test.groupby(['pmid'])['score'].rank(ascending=False)
    test.loc[:,'th']=np.where(test.loc[:,'pmid'].map(master_info['set']).isin([7,11,6,1]),th1,th2)
    test.loc[:,'predict1']=test.loc[:,'score']>test.loc[:,'th']
    test.loc[:,'predict2']=test.loc[:,'score_rank']==1
    test.loc[:,'predict']=test.loc[:,['predict1','predict2']].max(axis=1)
    test=test.sort_values(by=['score'],ascending=False)

    # Make final submission
    output=test[test.loc[:,'predict']]
    print th, len(output), len(output['pmid'].drop_duplicates())
    output['pmid_citation']="'"+output['pmid_citation'].astype(str)+"'"
    output=output.groupby(['pmid'])['pmid_citation'].apply(lambda x: '['+','.join(x)+']')
    df_subm['ref_list']=df_subm['pmid'].map(output).fillna('['+df_subm['pmid'].astype(str)+']')
    df_subm[['pmid','ref_list']].to_csv('02.Submission/10.Cutoff_%.03f_%.03f.csv'%(th1,th2),index=False)


In [ ]:
# Print model performance
print roc_auc_score(train['flag_citation'],clf.transform(train[idv])[:,1])
print roc_auc_score(valid['flag_citation'],clf.transform(valid[idv])[:,1])
print pd.Series(clf.feature_importances_,index=idv).sort_values(ascending=False)


In [ ]:
for th in [0.1,0.2,0.225,0.25,0.275,0.3,0.35,0.4,0.5,0.6,0.7,0.8,0.9]:
    # Make selection of recommendation
    test.loc[:,'score']=clf.transform(test.loc[:,idv].fillna(0))[:,1]
    test.loc[:,'score_rank']=test.groupby(['pmid'])['score'].rank(ascending=False)
    test.loc[:,'predict1']=test.loc[:,'score']>th
    test.loc[:,'predict2']=test.loc[:,'score_rank']==1
    test.loc[:,'predict']=test.loc[:,['predict1','predict2']].max(axis=1)
    test=test.sort_values(by=['score'],ascending=False)

    # Make final submission
    output=test[test.loc[:,'predict']]
    print th, len(output), len(output['pmid'].drop_duplicates())
    output['pmid_citation']="'"+output['pmid_citation'].astype(str)+"'"
    output=output.groupby(['pmid'])['pmid_citation'].apply(lambda x: '['+','.join(x)+']')
    df_subm['ref_list']=df_subm['pmid'].map(output).fillna('['+df_subm['pmid'].astype(str)+']')
    df_subm[['pmid','ref_list']].to_csv('02.Submission/08.Cutoff_%.03f.csv'%th,index=False)


In [ ]:
sample=master[master['flag_citation']==1]
print len(sample),len(sample['pmid'].drop_duplicates())

In [ ]:
train=master[master['sample']=='train']
valid=master[master['sample']=='valid']
test=master[master['sample']=='test']


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
idv=['f_num_common_authors','f_date_diff','f_max_article_title','f_sum_article_title','f_cnt_article_title',\
     'f_max_abstract','f_sum_abstract','f_cnt_abstract']
     # 'f_max_abstract_article_title','f_sum_abstract_article_title','f_cnt_abstract_article_title'
idv=[x for x in train.columns if x[:2]=='f_']
clf=DecisionTreeClassifier(max_depth=2, min_samples_split=200, min_samples_leaf=100,random_state=1234, class_weight={0:10,1:1})
clf.fit(train[idv],train['flag_citation'])

print roc_auc_score(train['flag_citation'],clf.transform(train[idv])[:,1])
print roc_auc_score(valid['flag_citation'],clf.transform(valid[idv])[:,1])
pd.Series(clf.feature_importances_,index=idv).sort_values(ascending=False)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
idv=['f_max_abstract_article_title_1','f_sum_article_title_1']
# idv=[x for x in train.columns if x[:2]=='f_']
clf=LogisticRegression(penalty='l1', solver='liblinear')
clf.fit(np.clip(train[idv],0,0.5),train['flag_citation'])

print roc_auc_score(train['flag_citation'],clf.predict(train[idv]))
print roc_auc_score(valid['flag_citation'],clf.predict(valid[idv]))
pd.Series(clf.coef_[0],index=idv).sort_values(ascending=False)


In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

dot_data = StringIO()

export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names=idv)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
train['score']=clf.transform(train[idv])[:,1]
sample=train[train['score']==train['score'].min()]
sample=sample[sample['flag_citation']==1]
sample.shape
sample

In [ ]:
master_info.loc[[12184798,9096352]]

In [ ]:
valid['score']=clf.transform(valid[idv].fillna(0))[:,1]

def simulation(th):
    valid.loc[:,'score_rank']=valid.groupby(['pmid'])['score'].rank(ascending=False)
    valid.loc[:,'predict1']=valid['score']>th
    valid.loc[:,'predict2']=valid['score_rank']==1
    valid.loc[:,'predict']=valid[['predict1','predict2']].max(axis=1)
    valid.loc[:,'NUM']=valid.loc[:,'predict']*valid.loc[:,'flag_citation']
    valid.loc[:,'DEN']=valid[['predict','flag_citation']].max(axis=1)
    scores=valid.groupby(['pmid'])[['NUM','DEN']].sum()
    return (scores['NUM']/scores['DEN']).mean()

[(x,simulation(x)) for x in np.arange(20,40)/100.0]

In [ ]:
pd.crosstab(valid['score']>0.275,valid['flag_citation'])

In [ ]:
(valid['score_rank']==1).value_counts()

In [ ]:
test.loc[:,'score']=clf.transform(test.loc[:,idv].fillna(0))[:,1]
test.loc[:,'score_rank']=test.groupby(['pmid'])['score'].rank(ascending=False)
test.loc[:,'predict1']=test.loc[:,'score']>0.275
test.loc[:,'predict2']=test.loc[:,'score_rank']==1
test.loc[:,'predict']=test.loc[:,['predict1','predict2']].max(axis=1)
test=test.sort_values(by=['score'],ascending=False)


In [ ]:
output=test[test.loc[:,'predict']]
output['pmid_citation']="'"+output['pmid_citation'].astype(str)+"'"
output=output.groupby(['pmid'])['pmid_citation'].apply(lambda x: '['+','.join(x)+']')
df_subm['ref_list']=df_subm['pmid'].map(output).fillna('['+df_subm['pmid'].astype(str)+']')

In [ ]:
# output=test.sort_values(by=['pmid','score'],ascending=[1,0])
# output=output[['pmid','pmid_citation','score']].drop_duplicates(subset=['pmid'])
# output.set_index('pmid',inplace=True)
# df_subm['pmid_citation']=df_subm['pmid'].map(output['pmid_citation']).fillna(df_subm['pmid'])
# df_subm['ref_list']=df_subm['pmid_citation'].map(lambda x: "['%d']"%x)


In [ ]:

df_subm[['pmid','ref_list']].to_csv('02.Submission/07.Cutoff_0.275.csv',index=False)

In [ ]:
'02.Submission/07.Cutoff_%.03f.csv'%0.275

In [ ]:
df_test.shape

In [ ]:
master_test['pmid'].drop_duplicates().shape